In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. Считаем данные

In [ ]:
X_train = pd.read_csv("./Data/x_train.csv", delimiter=';', header=None)
y_train = pd.read_csv("./Data/y_train.csv", delimiter=';', header=None)
X_test = pd.read_csv("./Data/x_test.csv", delimiter=';', header=None) 
y_test = pd.read_csv("./Data/y_test.csv", delimiter=';', header=None)

In [ ]:
X_train.head()

In [ ]:
# Сбалансированы ли данные?

In [ ]:
y_train = np.array(y_train)

### 2. Визуализация

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [ ]:
tsne = TSNE(verbose=1)
manifold = tsne.fit_transform(X_train)

In [ ]:
targets = np.unique(y_train)
cdict = {0: 'red', 1: 'blue', 2: 'green', 3: 'yellow', 4: 'black'}
plt.figure(figsize=(10, 10))
for t in targets:
    idx = np.where(y_train==t)[0]
    plt.scatter(manifold[idx, 0], manifold[idx, 1], c=cdict[t], s=10, alpha=0.5, label=t)
plt.legend()
plt.show()

In [ ]:
pca = PCA(n_components=2)
manifold = pca.fit_transform(X_train)

In [ ]:
targets = np.unique(y_train)
cdict = {0: 'red', 1: 'blue', 2: 'green', 3: 'yellow', 4: 'black'}
plt.figure(figsize=(10, 10))
for t in targets:
    idx = np.where(y_train==t)[0]
    plt.scatter(manifold[idx, 0], manifold[idx, 1], c=cdict[t], s=10, alpha=0.5, label=t)
plt.legend()
plt.show()

Забыл сказать... Оба метода работают на данных, которые центрированы и шкалированы

In [ ]:
#scaling here

In [ ]:
tsne = TSNE(verbose=1)
manifold = tsne.fit_transform(X_train)

In [ ]:
targets = np.unique(y_train)
cdict = {0: 'red', 1: 'blue', 2: 'green', 3: 'yellow', 4: 'black'}
plt.figure(figsize=(10, 10))
for t in targets:
    idx = np.where(y_train==t)[0]
    plt.scatter(manifold[idx, 0], manifold[idx, 1], c=cdict[t], s=10, alpha=0.5, label=t)
plt.legend()
plt.show()

In [ ]:
pca = PCA(n_components=2)
manifold = pca.fit_transform(X_train)

In [ ]:
targets = np.unique(y_train)
cdict = {0: 'red', 1: 'blue', 2: 'green', 3: 'yellow', 4: 'black'}
plt.figure(figsize=(10, 10))
for t in targets:
    idx = np.where(y_train==t)[0]
    plt.scatter(manifold[idx, 0], manifold[idx, 1], c=cdict[t], s=10, alpha=0.5, label=t)
plt.legend()
plt.show()

### 3. Посмотрим на RF на сырых данных

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def get_cv_score(X, Y, cv=5):
    rf_clf = RandomForestClassifier(n_estimators=100)
    scores = cross_val_score(rf_clf, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
    print("Accuracy: {0}".format(np.mean(scores)))

In [ ]:
get_cv_score(X_train, y_train)

### 3. Feature processing

In [ ]:
X = X_train.copy()
y = y_train.copy()

In [ ]:
def plot_importance(clf, X):
    importances = clf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]

    plt.figure(figsize=(20, 8))
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), X.columns[indices])
    plt.xlim([-1, X.shape[1]])
    plt.show()

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100).fit(X, y)
plot_importance(rf_clf, X)

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
#отберем признаки на основе важностей для random forest

In [ ]:
get_cv_score(X_sfm, y)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# Create the RFE object and compute a cross-validated score.
rf_clf = RandomForestClassifier(n_estimators=100)
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=rf_clf, step=1, cv=StratifiedKFold(2),
              scoring='accuracy')
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
X_rfecv = rfecv.transform(X)

In [ ]:
get_cv_score(X_rfecv, y)

In [ ]:
pca = PCA()
X_pca = pca.fit_transform(X)

In [ ]:
plt.plot(pca.explained_variance_ratio_)
pca.explained_variance_ratio_

In [ ]:
get_cv_score(X_pca, y)

In [ ]:
get_cv_score(X_pca[:, :6], y)

In [ ]:
plt.bar(range(X.shape[1]), abs(pca.components_[0]))